In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_train_NT=pd.read_csv('N_train.csv').drop("Unnamed: 0",axis=1)
df_test_NT=pd.read_csv('N_test.csv').drop("Unnamed: 0",axis=1)
df_target=pd.read_csv('target.csv').drop("Unnamed: 0",axis=1)
df_train=pd.read_csv('train_T.csv').drop("Unnamed: 0",axis=1)
df_train=df_train.drop('SalePrice',axis=1)
df_test=pd.read_csv('test_T.csv').drop("Unnamed: 0",axis=1)

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [4]:
params={'alpha':[1e-3,1e-2,1e-1,0.5,0.2,0.05,1,10,100]}

In [5]:
lasso_regressor=GridSearchCV(Lasso(),params,scoring='neg_mean_squared_error',cv=5)

In [6]:
lasso_regressor.fit(df_train_NT.values,df_target.values)

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1, 0.5, 0.2, 0.05, 1, 10,
                                   100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [7]:
lasso_regressor.best_estimator_

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [8]:
feat_sel_model=SelectFromModel(Lasso(alpha=0.005,random_state=0))

In [9]:
feat_sel_model.fit(df_train.values,df_target.values)

SelectFromModel(estimator=Lasso(alpha=0.005, copy_X=True, fit_intercept=True,
                                max_iter=1000, normalize=False, positive=False,
                                precompute=False, random_state=0,
                                selection='cyclic', tol=0.0001,
                                warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [10]:
bool_feat=feat_sel_model.get_support()
df_train_FT=df_train[df_train.columns[bool_feat]]
df_test_FT=df_test[df_test.columns[bool_feat]]

In [31]:
x_train=df_train_FT.values
y_train=df_target.values
x_test=df_test_FT.values

In [13]:
import xgboost

In [41]:
reg=xgboost.XGBRegressor(gamma=0.001,max_depth=6,objective='reg:squarederror')
reg

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.001,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [34]:
gamma=[0.001,0.01,0.1,1,10,100]
booster=['gbtree','gblinear']
learning_rate=[0.1,0.2,0.3]
max_depth=[3,4,5,6,7,8,9,10]
random_state=0
params={'gamma':gamma,'booster':booster,'learning_rate':learning_rate,'max_depth':max_depth}

In [35]:
hyp_regressor=GridSearchCV(reg,params,scoring='neg_mean_squared_error',cv=5)

In [36]:
hyp_regressor.fit(x_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    random_state=0, reg_alpha=...eg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'booster': ['gbtree', 'gblinear'],
                         'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
 

In [40]:
hyp_regressor.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.001,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [42]:
reg.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.001,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [45]:
y_predict_test=reg.predict(x_test)
y_predict_train=reg.predict(x_train)

In [46]:
from sklearn.metrics import mean_squared_error

In [50]:
mean_squared_error(y_train,y_predict_train)

0.9841120892437595

In [55]:
y_predict_test=np.exp(np.where(y_predict_test==0,0,y_predict_test))

In [67]:
sub=pd.DataFrame(np.round(y_predict_test,2),columns=['SalePrice'])

In [77]:
sub.set_index(np.arange(1461,2920),inplace=True)
sub.to_csv('kaggle2.csv',float_format='%.2f')

PermissionError: [Errno 13] Permission denied: 'kaggle2.csv'